### Predict price using neural network

#### We have 3 files
-- imports-85.data ## This file have all the data in csv format<br>-- imports-85.names ## This file have all column names, description and range of values<br>
-- misc
<br><br>
Data is taken from the UCI site. <br>
Site url: https://archive.ics.uci.edu/ml/datasets/Automobile

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

Using TensorFlow backend.


In [2]:
# all column name list
column_names = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base", "length", "width", "height", "curb-weight", "engine-type", "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]

In [3]:
# import data from csv
df = pd.read_csv("imports-85.data", names=column_names, header=None)

In [4]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [5]:
# All the nan values are given as '?'. So replaced all the '?' with nan
df[column_names] = df[column_names].replace({'?':np.nan})

In [6]:
df.isna().sum()

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64

In [7]:
#Price is dependent attribute so removing all the records which have nan price
df.dropna(subset = ["price"], inplace=True)

In [8]:
#filled all the nan field with ffill
# most of the numerical fields are casted as object because of '?' so can't replace with mean and median
df.fillna(method='ffill', inplace=True)

In [9]:
# first 3 records of normalized-losses was nan so it was not replaced in ffill, so replaced with 164
df['normalized-losses'].fillna(164, inplace=True)

In [10]:
# Updating the data type of all the fields
convert_dict = {'symboling': int, 
                'normalized-losses': int,
                'make': object,
                'fuel-type': object,
                'aspiration': object,
                'num-of-doors': object,
                'body-style': object,
                'drive-wheels': object,
                'engine-location': object,
                'wheel-base': float,
                'length': float,
                'width': float,
                'height': float,
                'curb-weight': int,
                'engine-type': object,
                'num-of-cylinders': object,
                'engine-size': int,
                'fuel-system': object,
                'bore': float,
                'stroke': float,
                'compression-ratio': float,
                'horsepower': int,
                'peak-rpm': int,
                'city-mpg': int,
                'highway-mpg': int,
                'price': float
            } 
df = df.astype(convert_dict)


In [11]:
# Replace number of door from categorical value to number(two, four)
#df['num-of-doors'].unique()
num_door_dict = {
    'two': 2,
    'four': 4
}
df['num-of-doors'] = df['num-of-doors'].map(num_door_dict)

In [12]:
# Replace number of Cylinder from categorical value to number
#'four', 'six', 'five', 'three', 'twelve', 'two', 'eight'
# df['num-of-cylinders'].unique()
num_of_cylinders_dict = {
    'four': 4,
    'six': 6,
    'five': 5,
    'three': 3,
    'twelve': 12,
    'two': 2,
    'eight': 8
}
df['num-of-cylinders'] = df['num-of-cylinders'].map(num_of_cylinders_dict)

In [13]:
# updated all categorical fields to number using one hot encoding for the below columns

# engine-location
#drive-wheels
#engine-type
#make
#aspiration
#body-style
#fuel-system
#fuel-type
for col_name in ['engine-location','drive-wheels', 'engine-type', 'make', 'aspiration', 'body-style', 'fuel-system', 'fuel-type']:
    df = pd.concat([df, pd.get_dummies(df[col_name])], axis=1)
    df.drop([col_name], axis='columns', inplace=True)

In [19]:
# Create train and target data from the data frame
train_data = df.sample(frac=0.85, random_state=200)
test_data = df.drop(train_data.index)

train_data_target = train_data['price']
train_data_params = train_data.drop(['price'],axis = 1)

test_data_target = test_data['price']
test_data_params = test_data.drop(['price'],axis = 1)

In [20]:
#Creating Neural network
model = Sequential()


# The Input Layer :
model.add(Dense(128, kernel_initializer='normal',input_dim = train_data_params.shape[1], activation='relu'))

# The Hidden Layers :
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               8704      
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 173,569
Trainable params: 173,569
Non-trainable params: 0
_________________________________________________________________


In [22]:
# fit data
model.fit(train_data_params, train_data_target, epochs=450, batch_size=32, validation_split = 0.2)


Train on 136 samples, validate on 35 samples
Epoch 1/450
136/136 [==============================] - 0s 485us/step - loss: 2246.7457 - mean_absolute_error: 2246.7454 - val_loss: 2087.3086 - val_mean_absolute_error: 2087.3086
Epoch 2/450
136/136 [==============================] - 0s 469us/step - loss: 2073.4852 - mean_absolute_error: 2073.4854 - val_loss: 2122.3620 - val_mean_absolute_error: 2122.3621
Epoch 3/450
136/136 [==============================] - 0s 461us/step - loss: 2272.5682 - mean_absolute_error: 2272.5684 - val_loss: 2108.9038 - val_mean_absolute_error: 2108.9038
Epoch 4/450
136/136 [==============================] - 0s 483us/step - loss: 2090.3555 - mean_absolute_error: 2090.3555 - val_loss: 2059.5615 - val_mean_absolute_error: 2059.5613
Epoch 5/450
136/136 [==============================] - 0s 544us/step - loss: 1982.7473 - mean_absolute_error: 1982.7472 - val_loss: 2060.8024 - val_mean_absolute_error: 2060.8025
Epoch 6/450
136/136 [==============================] - 0s 47

136/136 [==============================] - 0s 572us/step - loss: 1937.3129 - mean_absolute_error: 1937.3130 - val_loss: 2073.7107 - val_mean_absolute_error: 2073.7107
Epoch 47/450
136/136 [==============================] - 0s 517us/step - loss: 1946.5411 - mean_absolute_error: 1946.5411 - val_loss: 2162.7177 - val_mean_absolute_error: 2162.7175
Epoch 48/450
136/136 [==============================] - 0s 395us/step - loss: 2132.9292 - mean_absolute_error: 2132.9292 - val_loss: 2052.5254 - val_mean_absolute_error: 2052.5254
Epoch 49/450
136/136 [==============================] - 0s 413us/step - loss: 1942.3155 - mean_absolute_error: 1942.3154 - val_loss: 2103.0805 - val_mean_absolute_error: 2103.0806
Epoch 50/450
136/136 [==============================] - 0s 442us/step - loss: 1933.2679 - mean_absolute_error: 1933.2679 - val_loss: 2142.7387 - val_mean_absolute_error: 2142.7385
Epoch 51/450
136/136 [==============================] - 0s 457us/step - loss: 1952.9216 - mean_absolute_error: 19

136/136 [==============================] - 0s 408us/step - loss: 1903.2767 - mean_absolute_error: 1903.2766 - val_loss: 2074.9160 - val_mean_absolute_error: 2074.9160
Epoch 92/450
136/136 [==============================] - 0s 448us/step - loss: 1887.2957 - mean_absolute_error: 1887.2957 - val_loss: 1984.0984 - val_mean_absolute_error: 1984.0984
Epoch 93/450
136/136 [==============================] - 0s 416us/step - loss: 1953.3486 - mean_absolute_error: 1953.3486 - val_loss: 1984.3103 - val_mean_absolute_error: 1984.3103
Epoch 94/450
136/136 [==============================] - 0s 419us/step - loss: 1897.8897 - mean_absolute_error: 1897.8898 - val_loss: 2016.7965 - val_mean_absolute_error: 2016.7964
Epoch 95/450
136/136 [==============================] - 0s 435us/step - loss: 1907.8797 - mean_absolute_error: 1907.8799 - val_loss: 1950.3794 - val_mean_absolute_error: 1950.3795
Epoch 96/450
136/136 [==============================] - 0s 439us/step - loss: 1961.0590 - mean_absolute_error: 19

136/136 [==============================] - 0s 510us/step - loss: 2014.3116 - mean_absolute_error: 2014.3115 - val_loss: 2330.4741 - val_mean_absolute_error: 2330.4741
Epoch 137/450
136/136 [==============================] - 0s 497us/step - loss: 1899.3778 - mean_absolute_error: 1899.3778 - val_loss: 1938.2012 - val_mean_absolute_error: 1938.2012
Epoch 138/450
136/136 [==============================] - 0s 396us/step - loss: 1909.3849 - mean_absolute_error: 1909.3849 - val_loss: 1882.0207 - val_mean_absolute_error: 1882.0208
Epoch 139/450
136/136 [==============================] - 0s 457us/step - loss: 1879.7586 - mean_absolute_error: 1879.7587 - val_loss: 1908.4505 - val_mean_absolute_error: 1908.4504
Epoch 140/450
136/136 [==============================] - 0s 538us/step - loss: 1871.6247 - mean_absolute_error: 1871.6248 - val_loss: 1992.9551 - val_mean_absolute_error: 1992.9551
Epoch 141/450
136/136 [==============================] - 0s 474us/step - loss: 1984.4526 - mean_absolute_erro

136/136 [==============================] - 0s 627us/step - loss: 1727.1182 - mean_absolute_error: 1727.1182 - val_loss: 1763.4145 - val_mean_absolute_error: 1763.4146
Epoch 182/450
136/136 [==============================] - 0s 600us/step - loss: 1730.7135 - mean_absolute_error: 1730.7135 - val_loss: 2487.8553 - val_mean_absolute_error: 2487.8555
Epoch 183/450
136/136 [==============================] - 0s 536us/step - loss: 2107.2231 - mean_absolute_error: 2107.2231 - val_loss: 2183.8847 - val_mean_absolute_error: 2183.8845
Epoch 184/450
136/136 [==============================] - 0s 511us/step - loss: 2151.9922 - mean_absolute_error: 2151.9922 - val_loss: 2196.7627 - val_mean_absolute_error: 2196.7627
Epoch 185/450
136/136 [==============================] - 0s 589us/step - loss: 1747.9797 - mean_absolute_error: 1747.9796 - val_loss: 1815.6894 - val_mean_absolute_error: 1815.6895
Epoch 186/450
136/136 [==============================] - 0s 527us/step - loss: 1824.6775 - mean_absolute_erro

136/136 [==============================] - 0s 427us/step - loss: 2015.5790 - mean_absolute_error: 2015.5791 - val_loss: 1734.2926 - val_mean_absolute_error: 1734.2925
Epoch 227/450
136/136 [==============================] - 0s 506us/step - loss: 1930.9623 - mean_absolute_error: 1930.9623 - val_loss: 1890.1688 - val_mean_absolute_error: 1890.1687
Epoch 228/450
136/136 [==============================] - 0s 479us/step - loss: 2042.9725 - mean_absolute_error: 2042.9724 - val_loss: 1834.2918 - val_mean_absolute_error: 1834.2917
Epoch 229/450
136/136 [==============================] - 0s 535us/step - loss: 1957.4926 - mean_absolute_error: 1957.4927 - val_loss: 1772.8769 - val_mean_absolute_error: 1772.8770
Epoch 230/450
136/136 [==============================] - 0s 463us/step - loss: 1662.3595 - mean_absolute_error: 1662.3595 - val_loss: 2398.3513 - val_mean_absolute_error: 2398.3513
Epoch 231/450
136/136 [==============================] - 0s 404us/step - loss: 2032.3918 - mean_absolute_erro

136/136 [==============================] - 0s 530us/step - loss: 1496.1634 - mean_absolute_error: 1496.1633 - val_loss: 1638.0170 - val_mean_absolute_error: 1638.0170
Epoch 272/450
136/136 [==============================] - 0s 511us/step - loss: 1514.7598 - mean_absolute_error: 1514.7599 - val_loss: 1835.4670 - val_mean_absolute_error: 1835.4669
Epoch 273/450
136/136 [==============================] - 0s 432us/step - loss: 1573.6220 - mean_absolute_error: 1573.6219 - val_loss: 2394.9146 - val_mean_absolute_error: 2394.9146
Epoch 274/450
136/136 [==============================] - 0s 439us/step - loss: 1901.3065 - mean_absolute_error: 1901.3065 - val_loss: 1816.7082 - val_mean_absolute_error: 1816.7083
Epoch 275/450
136/136 [==============================] - 0s 448us/step - loss: 1538.2474 - mean_absolute_error: 1538.2474 - val_loss: 2148.3792 - val_mean_absolute_error: 2148.3792
Epoch 276/450
136/136 [==============================] - 0s 435us/step - loss: 1433.9472 - mean_absolute_erro

136/136 [==============================] - 0s 440us/step - loss: 1390.0822 - mean_absolute_error: 1390.0822 - val_loss: 2248.1323 - val_mean_absolute_error: 2248.1323
Epoch 317/450
136/136 [==============================] - 0s 432us/step - loss: 1438.2908 - mean_absolute_error: 1438.2908 - val_loss: 1578.3007 - val_mean_absolute_error: 1578.3007
Epoch 318/450
136/136 [==============================] - 0s 408us/step - loss: 1502.6447 - mean_absolute_error: 1502.6448 - val_loss: 1559.1236 - val_mean_absolute_error: 1559.1237
Epoch 319/450
136/136 [==============================] - 0s 406us/step - loss: 1385.3589 - mean_absolute_error: 1385.3588 - val_loss: 2045.1468 - val_mean_absolute_error: 2045.1469
Epoch 320/450
136/136 [==============================] - 0s 420us/step - loss: 1422.5539 - mean_absolute_error: 1422.5538 - val_loss: 1601.7805 - val_mean_absolute_error: 1601.7805
Epoch 321/450
136/136 [==============================] - 0s 399us/step - loss: 1254.7140 - mean_absolute_erro

136/136 [==============================] - 0s 459us/step - loss: 1236.4344 - mean_absolute_error: 1236.4344 - val_loss: 1894.6992 - val_mean_absolute_error: 1894.6993
Epoch 362/450
136/136 [==============================] - 0s 430us/step - loss: 1836.1140 - mean_absolute_error: 1836.1140 - val_loss: 1829.2582 - val_mean_absolute_error: 1829.2583
Epoch 363/450
136/136 [==============================] - 0s 481us/step - loss: 1538.8857 - mean_absolute_error: 1538.8857 - val_loss: 1711.2511 - val_mean_absolute_error: 1711.2511
Epoch 364/450
136/136 [==============================] - 0s 482us/step - loss: 1282.2400 - mean_absolute_error: 1282.2400 - val_loss: 1599.2706 - val_mean_absolute_error: 1599.2706
Epoch 365/450
136/136 [==============================] - 0s 592us/step - loss: 1207.5344 - mean_absolute_error: 1207.5343 - val_loss: 1762.6249 - val_mean_absolute_error: 1762.6249
Epoch 366/450
136/136 [==============================] - 0s 489us/step - loss: 1191.8361 - mean_absolute_erro

136/136 [==============================] - 0s 526us/step - loss: 1167.1114 - mean_absolute_error: 1167.1113 - val_loss: 1675.3609 - val_mean_absolute_error: 1675.3608
Epoch 407/450
136/136 [==============================] - 0s 437us/step - loss: 1213.0321 - mean_absolute_error: 1213.0321 - val_loss: 1580.7290 - val_mean_absolute_error: 1580.7290
Epoch 408/450
136/136 [==============================] - 0s 421us/step - loss: 1110.6279 - mean_absolute_error: 1110.6279 - val_loss: 2183.8270 - val_mean_absolute_error: 2183.8269
Epoch 409/450
136/136 [==============================] - 0s 451us/step - loss: 1343.1248 - mean_absolute_error: 1343.1248 - val_loss: 1560.5846 - val_mean_absolute_error: 1560.5846
Epoch 410/450
136/136 [==============================] - 0s 439us/step - loss: 1083.0259 - mean_absolute_error: 1083.0260 - val_loss: 1513.9084 - val_mean_absolute_error: 1513.9083
Epoch 411/450
136/136 [==============================] - 0s 441us/step - loss: 1163.9786 - mean_absolute_erro

In [27]:
model.predict(test_data_params.iloc[[2]])

array([[31217.828]], dtype=float32)

In [28]:
test_data_target.iloc[[2]]

15    30760.0
Name: price, dtype: float64